In [1]:
import requests
import pandas as pd
import time

class PipefyAuth:
    def __init__(self, client_id, client_secret):
        self.client_id = client_id
        self.client_secret = client_secret
        self.token_url = "https://app.pipefy.com/oauth/token"
        self.access_token = None
        self.token_expiry = None
    
    def get_token(self):
        if self.access_token and self.token_expiry and time.time() < self.token_expiry:
            return self.access_token
        
        data = {
            "grant_type": "client_credentials",
            "client_id": self.client_id,
            "client_secret": self.client_secret
        }
        
        try:
            response = requests.post(self.token_url, json=data)
            response.raise_for_status()
            token_data = response.json()
            self.access_token = token_data["access_token"]
            self.token_expiry = time.time() + token_data["expires_in"]
            return self.access_token
        except requests.exceptions.RequestException as e:
            print(f"Erro ao obter token: {e}")
            return None


client_id = "RbxhzypoqBfiV9ZlpX-56wJltoom0C_hot6_wcGppoQ"
client_secret = "w3tdyxrxaubuqwN0EqNlW3cXD5ObSplxqA21wB9Ed9Q"

auth = PipefyAuth(client_id, client_secret)

PIPE_ID = 305611585
PIPE_REPORT_ID = 300927833
GRAPHQL_URL = "https://api.pipefy.com/graphql"

def iniciar_exportacao(auth):
    token = auth.get_token()
    if not token:
        return None

    query = f"""
    mutation {{
      exportPipeReport(input: {{
        pipeId: {PIPE_ID},
        pipeReportId: {PIPE_REPORT_ID}
      }}) {{
        pipeReportExport {{
          id
        }}
      }}
    }}
    """
    response = requests.post(GRAPHQL_URL, headers={
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json"
    }, json={"query": query})
    resp = response.json()
    print("Início da exportação:", resp)
    try:
        return resp['data']['exportPipeReport']['pipeReportExport']['id']
    except:
        print("Erro ao iniciar exportação:", resp)
        return None

def verificar_status(auth, export_id):
    token = auth.get_token()
    if not token:
        return None

    query = f"""
    {{
        pipeReportExport(id: {export_id}) {{
            fileURL
            finishedAt
        }}
    }}
    """
    response = requests.post(GRAPHQL_URL, headers={
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json"
    }, json={"query": query})
    resp = response.json()
    try:
        export = resp['data']['pipeReportExport']
        if export['finishedAt']:
            # Baixar
            r = requests.get(export['fileURL'])
            filename = 'relatorio.xlsx'
            with open(filename, 'wb') as f:
                f.write(r.content)
            print("Baixado:", filename)
            return filename
        else:
            print("Ainda não pronto.")
            return None
    except:
        print("Erro ao verificar:", resp)
        return None

def main():
    export_id = iniciar_exportacao(auth)
    if not export_id:
        print("Falha ao iniciar.")
        return
    print(f"ID da exportação: {export_id}")

    while True:
        arquivo = verificar_status(auth, export_id)
        if arquivo:
            break
        print("Aguardando 5s...")
        time.sleep(5)

    # Carregar no pandas
    if arquivo.endswith('.csv'):
        df = pd.read_csv(arquivo)
    else:
        df = pd.read_excel(arquivo)
    return df

if __name__ == "__main__":
    df = main()
    print(df)

Início da exportação: {'data': {'exportPipeReport': {'pipeReportExport': {'id': '314740690'}}}}
ID da exportação: 314740690
Ainda não pronto.
Aguardando 5s...
Baixado: relatorio.xlsx
                                                  Title       Current phase  \
0                         BM001440 - H.DR. F. MORAN - V     Arquivados Maio   
1                         BM001006 - H.DR. F. MORAN - V    Arquivados Abril   
2                                BM000827 - HU-UFGD - L    Arquivados Março   
3                            BH001601 - SANTA MARIA - L    Arquivados Junho   
4     SP001562 - LWM MATERIAIS MEDICOS E CIRURGICOS ...    Arquivados Junho   
...                                                 ...                 ...   
1995  BM000939 - HOSPITAL DAS CLINICAS DA FACULDADE ...    Arquivados Abril   
1996                      BM002364 - H.DR. F. MORAN - V   Arquivados Agosto   
1997                        SP0749 - INST. MAIS SAUDE-V  Arquivados Janeiro   
1998                       

In [2]:
df

,Title,Current phase,Cliente,Representante,Representante Secundário,Etiquetas,Itens do Pedido,CNPJ emissor de NF,Prazo de Pagamento,Tipo de NF,Número da NF,Data de Faturamento,Valor Faturado,Frete,Valor AVIAT - ( Frete incluso na nota),Valor AVIAT ( Frete não cobrado do cliente),Valor Correios ( Frete incluso na nota),Valor Correios - ( Frete não cobrado do cliente)
0,BM001440 - H.DR. F. MORAN - V,Arquivados Maio,61.699.567/0018-30,Michele Rosete,NaN,"Brasil MED, Somente faturamento",SLING-IUF,Brasil Med - 31.282.018/0001-57,30 dias,Venda,1884,2025-05-28,900.0,NaN,NaN,NaN,NaN,NaN
1,BM001006 - H.DR. F. MORAN - V,Arquivados Abril,61.699.567/0018-30,Michele Rosete,NaN,"Brasil MED, Somente faturamento",SLING-IUF,Brasil Med - 31.282.018/0001-57,30 dias,Venda,1751,2025-04-14,900.0,NaN,NaN,NaN,NaN,NaN
2,BM000827 - HU-UFGD - L,Arquivados Março,15126437000909,Noemia Sattin,NaN,Brasil MED,"PEG-20, PEG-24",Brasil Med - 31.282.018/0001-57,30 dias,Venda,1684,2025-03-26,11650.0,AVIAT ( Frete não cobrado do cliente),NaN,NaN,NaN,NaN
3,BH001601 - SANTA MARIA - L,Arquivados Junho,44720530000180,Ricardo santos,NaN,Unidas BH,GT*-2820,Unidas BH - 17.094.914/0001-61,30 dias,Venda,10699,2025-06-16,1200.0,Correios ( Frete não cobrado do cliente),NaN,NaN,NaN,NaN
4,SP001562 - LWM MATERIAIS MEDICOS E CIRURGICOS ...,Arquivados Junho,48622250000190,Michele Rosete,NaN,"Pendência de CADASTRO CLIENTE, Unidas SP",SLING-IUF*,Unidas SP - 17.094.914/0002-42,30 dias,Venda,1892,2025-06-12,5440.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,BM000939 - HOSPITAL DAS CLINICAS DA FACULDADE ...,Arquivados Abril,12.474.705/0001-20,Ricardo santos,Melina Basso,Brasil MED,BLDS-14536*,Brasil Med - 31.282.018/0001-57,30 dias,Venda,1711,2025-04-07,12600.0,Correios ( Frete não cobrado do cliente),NaN,NaN,NaN,NaN
1996,BM002364 - H.DR. F. MORAN - V,Arquivados Agosto,61.699.567/0018-30,Michele Rosete,NaN,"Brasil MED, Somente faturamento",SLING-IUF*,Brasil Med - 31.282.018/0001-57,30 dias,Venda,2160,2025-08-25,900.0,NaN,NaN,NaN,NaN,NaN
1997,SP0749 - INST. MAIS SAUDE-V,Arquivados Janeiro,18.963.002/0011-13,Vanessa Catto,NaN,"Bionexo, Unidas SP",DSB-0413,Unidas SP - 17.094.914/0002-42,30 dias,Venda,1007,2025-01-20,1300.0,NaN,NaN,NaN,NaN,NaN
1998,BM000714 - FSNH - L,Arquivados Abril,11055682000156,Noemia Sattin,NaN,Brasil MED,SLVE,Brasil Med - 31.282.018/0001-57,30 dias,Venda,1730,2025-04-09,3898.0,Correios ( Frete não cobrado do cliente),NaN,NaN,NaN,NaN
